# Cross-Validation for KNN

### Import Packages

In [11]:
import pandas as pd
import numpy as np
import os 

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Step 1: Build DataFrame and Define ML Problem

We will work with the "cell2celltrain" data set. This data set is already preprocessed, with the proper formatting, outliers, and missing values taken care of, and all numerical columns scaled to the [0, 1] interval. One-hot encoding has been performed on all categorical columns. It is ready for modeling.

In [13]:
filename = os.path.join(os.getcwd(), "..", "..", "data", "cell2celltrain_cv.csv")
df = pd.read_csv(filename, header=0)

#### Define the Label

This is a binary classification problem in which we will predict customer churn. The label is the `Churn` column.

#### Identify Features

Our features will be all of the remaining columns in the dataset.

## Step 2. Create Labeled Examples from the Data Set 

Let's obtain columns from our data set to create labeled examples. 
The code cell below carries out the following steps:

* Gets the `Churn` column from DataFrame `df` and assigns it to the variable `y`. This is our label.
* Gets all other columns from DataFrame `df` and assigns them to the variable `X`. These are our features. 


In [14]:
y = df['Churn']
X = df.drop(columns = 'Churn', axis=1)

## Step 3: Create Training and Test Data Sets

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1234)

###  Inspect the Training and Test Data Sets

In [16]:
print(X_train.shape)
print(X_test.shape)

(45942, 84)
(5105, 84)


In [17]:
X_train.head()

,CustomerID,ChildrenInHH,HandsetRefurbished,HandsetWebCapable,TruckOwner,RVOwner,HomeownershipKnown,BuysViaMailOrder,RespondsToMailOffers,OptOutMailings,...,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student,Married_False,Married_True,Married_nan
45106,3356966,False,False,True,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
38896,3310250,True,False,True,False,False,True,True,True,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
29853,3237338,False,False,True,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
33048,3263222,False,False,True,False,False,True,True,True,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
21061,3165118,False,False,True,False,False,False,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Step 4: Train a KNN Classifier and Perform k-Fold Cross-Validation


We just created a training set and a test set. We won't use our test set in this exercise, but we will focus on the training set.

We will perform k-fold cross-validation on our training set. K-fold cross-validation is an out-of-sample technique that helps estimate how well your model generalizes to new data.

k-fold cross-validation splits training data into equally sized subsets, or folds (k). We train and test 'k' times, such that each time, we train on k-1 *training* folds and test on 1 *validation* fold. Therefore, every fold will have a chance to serve as a validation set. We then average the resulting accuracies obtained on each of the k iterations to determine the accuracy of the model.

We will use built-in cross-validation tools from `sklearn` that will help us perform a k-fold cross-validation on our KNN model. 

First, let's create the KNN model object.

### a. Create the KNN Model Object

In [18]:
# Create the  KNeighborsClassifier model object 
model = KNeighborsClassifier(3)

### b. Train and Evaluate the Model's Performance Using Two K-fold Cross-Validation Approaches



### Scikit-Learn's `KFold` Approach

First we will use scikit-learn's `KFold` class. This class helps us partition our training data set into k folds. Let's import the `KFold` class from `sklearn`:

In [19]:
from sklearn.model_selection import KFold

The code cell below is performing 5-fold cross-validation. We use `KFold` to partition our training data (`X_train` and `y_train`) into 5 folds: `KFold` separates our training data into 5 subsets of rows. We then train our KNN model on each subset and evaluate its performance. Therefore, we will train and evaluate our KNN model for a total of 5 times - once per data subset. (Note that we won't change the hyperparameters of our model; our KNN model, with its current configuration of 3 nearest neighbors, will be trained 5 times using different training data subsets.) We will then average the model's accuracy score at each iteration to compute the overall model accuracy. 

Examine the code cell below. It is doing the following:

1. Splits the training set into 5 subsets
2. Loops over each subset, and for each data subset it:
    1. Splits the data subset into new training and validation sets
    2. Trains the KNN model on the new training set
    3. Tests the model on the new validation set and computes the accuracy of the model's predictions.
    4. Saves the accuracy score to a list.
3. Computes the average of the 5 accuracy scores.

In [20]:
num_folds = 5
folds = KFold(n_splits = num_folds, random_state=None)

acc_scores = []

for train_row_index , test_row_index in folds.split(X_train): 
    
    # our new partition of X_train and X_val
    X_train_new  = X_train.iloc[train_row_index] 
    X_val = X_train.iloc[test_row_index]
    
    # our new partition of y_train and y_val
    y_train_new = y_train.iloc[train_row_index]
    y_val = y_train.iloc[test_row_index]
    
    model.fit(X_train_new, y_train_new)
    predictions = model.predict(X_val)
     
    iteration_accuracy = accuracy_score(predictions , y_val)
    acc_scores.append(iteration_accuracy)
     
        
for i in range(len(acc_scores)):
    print('Accuracy score for iteration {0}: {1}'.format(i+1, acc_scores[i]))

avg_scores = sum(acc_scores)/num_folds
print('\nAverage accuracy score: {}'.format(avg_scores))


Accuracy score for iteration 1: 0.6284688214169115
Accuracy score for iteration 2: 0.6305365110458157
Accuracy score for iteration 3: 0.6320200261210275
Accuracy score for iteration 4: 0.6238572050500653
Accuracy score for iteration 5: 0.630387461906835

Average accuracy score: 0.629054005108131


###  Scikit-Learn's `cross_val_score` Approach

Scikit-learn has another method of performing k-fold cross-validation that performs the steps above in one line of code. This is the `cross_val_score()` function.  It allows you to specify the number of folds 'k'. It automatically and repeatedly subsamples the training and validation sets out of the initial training data and computes the accuracy scores when testing on these validation sets. It returns one accuracy score per iteration ('k' iterations in total). 


First let's import the function from `sklearn`:

In [21]:
from sklearn.model_selection import cross_val_score

In [22]:
# Use function cross_val_score() to perform 5-fold cross-validation on the training data and 
# save the result to variable accuracy_scores

print('Running Cross-Validation...')

accuracy_scores = cross_val_score(model, X_train, y_train, cv=5)

# Print the accuracy scores
print('Accuracies for the five training/test iterations on the validation sets:')
print(accuracy_scores)


Running Cross-Validation...
Accuracies for the five training/test iterations on the validation sets:
[0.62901295 0.63075416 0.63006095 0.62483674 0.62940792]


The `cross_val_score()` function returns a NumPy array. In the code cell below, use NumPy's `mean()` and `std()` methods on array `accuracy_scores` to find the average accuracy score for our model and to see the degree of variance among each score.

Save the mean to variable `acc_mean` and save the standard deviation to the variable `acc_std`.

In [23]:
# Find the mean accuracy score and save to variable 'acc_mean'

acc_mean = accuracy_scores.mean()
print('The mean accuracy score across the five iterations:')
print(acc_mean)

# Find the standard deviation of the accuracy score and save to variable 'acc_std'
acc_std = accuracy_scores.std()

# Print the standard deviation of the accuracy scores using the std() method to see the degree of variance.
print('The standard deviation of the accuracy score across the five iterations:')
print(acc_std)


The mean accuracy score across the five iterations:
0.6288145457746441
The standard deviation of the accuracy score across the five iterations:
0.0020750906566908206


Notice that the five resulting accuracy scores are good, and the standard deviation among the scores are low, indicating that our model performs well. 